In [1]:
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [2]:
torch.manual_seed(0)

In [3]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='home/cifar10/', train=True,download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=8,shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=8,shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [50]:
class CNNModule(nn.Module):
    def __init__(self):
        super(CNNModule, self).__init__()
        self.conv1 = nn.Conv2d(3, 9, 5,padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(9, 27, 5)
        self.conv3 = nn.Conv2d(27, 54, 5)
        #self.conv4 = nn.Conv2d(54, 108, 5) 
        #self.conv5 = nn.Conv2d(108, 216, 5) 
        self.fc1 = nn.Linear(54 * 1 * 1, 180) #Size of filter*height*width
        self.fc2 = nn.Linear(180, 64)
        self.fc3 = nn.Linear(64, 10)
        self.batch1 = nn.BatchNorm2d(9)
        self.batch2 = nn.BatchNorm2d(27)
        self.batch3 = nn.BatchNorm2d(54)
        self.batch4 = nn.BatchNorm2d(108)
        #self.batch5 = nn.BatchNorm2d(216)

    def forward(self, x):
        out = self.pool(F.relu(self.batch1(self.conv1(x))))
        out = self.pool(F.relu(self.batch2(self.conv2(out))))
        out = self.pool(F.relu(self.batch3(self.conv3(out))))
        #out = self.pool(F.relu(self.batch4(self.conv4(out))))
        #out = self.pool(F.relu(self.batch5(self.conv5(out))))
        #print(out.shape)
        #x = self.pool(F.relu(self.batch4(self.conv4(x))))
        out = out.view(out.size(0), -1) #Size of filter*height*width
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [51]:
cnn = CNNModule()

In [54]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum = 0.9)

In [57]:
start_time = time.time()
for iteration in range(10):  # loop over the dataset multiple times

    iterative_loss = 0.0
    for i, (inputs,labels) in enumerate(trainloader, 0):

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        iterative_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (iteration + 1, i + 1, iterative_loss / 1000))
            # print(loss.item())
            iterative_loss = 0.0

print('Finished Training')
print("Time taken to train is %s seconds" % (time.time() - start_time))

[1,  1000] loss: 0.752
[1,  2000] loss: 0.770
[1,  3000] loss: 0.805
[1,  4000] loss: 0.822
[1,  5000] loss: 0.787
[1,  6000] loss: 0.831
[2,  1000] loss: 0.758
[2,  2000] loss: 0.753
[2,  3000] loss: 0.792
[2,  4000] loss: 0.769
[2,  5000] loss: 0.790
[2,  6000] loss: 0.800
[3,  1000] loss: 0.721
[3,  2000] loss: 0.754
[3,  3000] loss: 0.773
[3,  4000] loss: 0.781
[3,  5000] loss: 0.778
[3,  6000] loss: 0.768
[4,  1000] loss: 0.737
[4,  2000] loss: 0.741
[4,  3000] loss: 0.758
[4,  4000] loss: 0.749
[4,  5000] loss: 0.751
[4,  6000] loss: 0.749
[5,  1000] loss: 0.696
[5,  2000] loss: 0.718
[5,  3000] loss: 0.720
[5,  4000] loss: 0.734
[5,  5000] loss: 0.761
[5,  6000] loss: 0.744
[6,  1000] loss: 0.692
[6,  2000] loss: 0.703
[6,  3000] loss: 0.724
[6,  4000] loss: 0.733
[6,  5000] loss: 0.715
[6,  6000] loss: 0.730
[7,  1000] loss: 0.667
[7,  2000] loss: 0.680
[7,  3000] loss: 0.688
[7,  4000] loss: 0.710
[7,  5000] loss: 0.717
[7,  6000] loss: 0.713
[8,  1000] loss: 0.674
[8,  2000] 

In [58]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 65 %
